# Scheduling API Calls

https://www.baeldung.com/linux/job-control-disown-nohup
    
https://unix.stackexchange.com/questions/3886/difference-between-nohup-disown-and

https://en.wikipedia.org/wiki/Cron

## Environment Variable WEBUI_PORT

Users of the lambda1 machine need to use their own port number when starting the the GUI to avoid having their output tangled up with that of other users. I edited each user's webui-user.sh file to include their port number when I setup the users.

In this notebook I will introduce several scripts for controlling the webui API, and two of those scripts need to know the user's port number.

We are going to add a line to each user's ~/.bashrc file to store their port number as an environment variable. It would be pretty easy for me to edit each user's file to make the change, but users need to know about the nano editor in order to schedule their jobs to run on the API, so I will demonstrate how this is done in class, and users can follow along.

### .bashrc


The file ~/.bashrc has start up commands for bash shell processes.

For example, the line `alias ll='ls -alF` creates the alias ll, which is short for `ls -alF`.

We need to add the `export WEBUI_PORT=XXXX`, where you will substitute your port number for XXXX.

### nano

nano is a basic text editor.  It is easy to use because the commands you need are shown at the bottom of the screen.

ssh to lambda1 and we will all edit our .bashrc files.

## Scripts

Your path contains the directory ~/.local/bin, so if you put a script in that directory, then you can call it by name.  There are three scripts in the directory /opt/models/scripts that you should copy to your ./local/bin directory.

**Commands**

>cd .local/bin
>
>cp /opt/models/scripts/\*.sh .
>
>cd
>
>ping.sh

## ping.sh

This should return `API is available` if your webui API is running, and `API needs to be started` if it is not.

## start.sh

<img src='../data/api-start.png'>

* start.sh 3 starts the webui.sh with flags --api and --device-id=3
* First call to ping said the API needs to be started. It was still starting up.

Here is the script:

In [3]:
!cat ../data/start.sh

#!/bin/bash

cd /home/${USER}/stable-diffusion-webui

if [ $# -gt 0 ]
then
	nohup ./webui.sh --api --device-id=$1 &
else
	nohup ./webui.sh --api &
fi

echo $!>process_id


## train_lora.sh

<img src='../data/api-train.png'>

Here is the script:

In [4]:
!cat ../data/train_lora.sh

#!/bin/bash

if [ $# -lt 1 ]
then
	echo "Supply your model name as an argument"
else
	myping="`curl -X 'GET' --silent --output /dev/null \
		--write-out '%{http_code}' \
		http://localhost:${WEBUI_PORT}/internal/ping`"

	if [ $myping -eq 200 ]
	then
		echo "Calling curl ..."
		url="http://localhost:${WEBUI_PORT}/dreambooth/start_training?model_name=${1}&use_tx2img=true"
        	echo $url

		resp="`curl -X 'POST' --silent $url -H 'accept: application/json' -d ''`"
		echo "Done with curl"
		echo $resp
	else
		echo "API needs to be started"
	fi
fi


## nohup.out

The start.sh script changed to the stable-diffusion directory and started the webui with nohup.

The output of the command is sent to the file nohup.out.

tail -f nohup.out will show the nohup.out file, updating as the file changes.

<img src='../data/api-tail.png'>

## Stop Webui

<img src='../data/api-stop.png'>

ps -ef shows details of running processes.

Pipe the output to grep and look for rows that contain launch.py.  The first one is the process we want to kill.

## Crontab

Demonstrate in class ...